# Tensorflow tutorial for CNN

## Import necessary module and method.

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

ModuleNotFoundError: No module named 'tensorflow'

## Convinient helper functions.

In [ ]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

## Open MNIST dataset and create a session

In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
sess = tf.InteractiveSession()

## Input
`(x,y_)` is the placeholder for training samples. Each image is `28x28`, so it is a `784`-vector. The number of classes is `10`.
`x` should be reshape to `[-1,28,28,1]`. Both `None` and `-1` mean `any` length.

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1,28,28,1])

## 1st convolutional layer
The 1st "layer" is `pool(relu(conv(x, w) + b)`. There are `32` `5x5`-filters. Each filter has a scalar bias. `28x28` image is `max-pooled` to `14x14` image. The output is a `[-1,14,14,32]`-tensor. Note that
  * For convolution, we have to specify FOUR dimensions for tensors: width, height, num_input_channels, num_filters(num_output_channels)
  * The input tensor should be 4-tensor: `(nx, ny, nz, ni)`
  * The weight tensor should also be 4-tensor: `(filter_width, filter_height, ni, nc)`
  * So when we compute convolution, the `num_image_channels`-dimension is reduced, the output became `(nx,ny,nz,nc)`-tensor
  * `conv(x,w)` is similar to `numpy.dot(x,w)`, the last dimension of `x` and the last but one dimension of `w` are reduced.

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## 2nd convolutional layer
The 2nd "layer" is also `poll(relu(conv(x,w) + b)`. The input is `[-1,14,14,32]`-tensor, and the output is `[-1,7,7,64]`-tensor.

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## 3rd full-connected layer
The 3rd "layer" is a `full-connected-relu` layer. The `[-1,7,7,64]`-tensor is reshaped to `[-1,7*7*64]`-tensor. The input is `[-1,7,7,64]`-tensor, and the output is `[-1,1024]`-tensor.

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## 4th dropout layer
The 4th "layer" is a `dropout` layer. With probability `keep_prob`, input elements are scaled by `1/keep_prob`; otherwise output `0`.

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## 5th full-connected output layer
The 5th "layer" is a `full-connected-softmax` layer. The input layer is `[-1,1024]`-tensor, the output is `[-1,10]`-tensor.

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## Cost function
Now, we need to compute the cost/error as `cross entropy`: $\frac1N\sum_i^N\sum_j^{10} y_{ij}\log \frac{1}{\hat y_{ij}}$, where $y$ is the probabilities of real classes, $\hat y$ is the predicted probabilities. Our target is to minimize the cross entropy, which is implemented by `Adam` algorithm.

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

There are also other derived varibles. `tf.argmax(y_conv,1)` is a vector of predicted class indices. `tf.argmax(y_,1)` is a vector of ground-truth class indices. So `correct_prediction` is a vector of bools to indicate which sample is correctly classfied. And `accuracy` is the ratio of correctly classfied samples.

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training
Now we initialize all tensorflow variables and start to train.

In [ ]:
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))